In [1]:
import dask
from dask.distributed import Client

In [1]:
# TODO: add login line
# coiled login --token XXX

In [2]:
import coiled

cluster = coiled.Cluster(n_workers=20)
client = cluster.get_client()

Output()

Output()

In [3]:
client

<Client: 'tls://10.0.41.117:8786' processes=11 threads=44, memory=163.30 GiB>

In [29]:
cluster.scale(10)

In [38]:
client.restart()

<Client: 'tls://10.0.41.117:8786' processes=0 threads=0, memory=0 B>

# Use Case: Data cleaning and exploration

Reading, formatting, writing dataset. Preparing it for additional downstream analaysis. 

In [4]:
%%time

import dask.dataframe as dd

ddf = dd.read_csv(
    "s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
    dtype={"wav_match_flag": "category"},
)

CPU times: user 47.6 s, sys: 1.18 s, total: 48.8 s
Wall time: 49.2 s


In [14]:
%%time

# Convert to datetime
ddf["on_scene_datetime"] = dd.to_datetime(ddf["on_scene_datetime"], format="mixed")
ddf["pickup_datetime"] = dd.to_datetime(ddf["pickup_datetime"], format="mixed")

# Unpack columns
ddf = ddf.assign(
    accessible_vehicle=ddf.on_scene_datetime.isnull(),
    pickup_month=ddf.pickup_datetime.dt.month,
    pickup_dow=ddf.pickup_datetime.dt.dayofweek,
    pickup_hour=ddf.pickup_datetime.dt.hour,
)
ddf = ddf.drop(columns=["on_scene_datetime", "pickup_datetime"])

# Format airport_fee
ddf["airport_fee"] = ddf["airport_fee"].replace("None", 0).astype(float).fillna(0)

# Remove outliers
lower_bound = 0
Q3 = ddf["trip_time"].quantile(0.75)
upper_bound = Q3 + (1.5 * (Q3 - lower_bound))
ddf = ddf.loc[(ddf["trip_time"] >= lower_bound) & (ddf["trip_time"] <= upper_bound)]

# # Categorize *_flag columns
# ddf = ddf.categorize(columns=["shared_request_flag", "shared_match_flag", "access_a_ride_flag", "wav_request_flag", "wav_match_flag"])

CPU times: user 130 ms, sys: 7.23 ms, total: 137 ms
Wall time: 135 ms


In [55]:
service_names = {
    "HV0002": "juno",
    "HV0005": "lyft",
    "HV0003": "uber",
    "HV0004": "via",
}

ddf["service_names"] = ddf["hvfhs_license_num"].map(service_names)
ddf["service_names"]

Dask Series Structure:
npartitions=9961
    string
       ...
     ...  
       ...
       ...
Name: service_names, dtype: string
Dask Name: getitem, 8 graph layers

In [56]:
ddf["service_names"].head()

542399    lyft
597094    lyft
642693    lyft
741280    uber
823961    uber
Name: service_names, dtype: string

In [54]:
ddf2.tips.describe().compute()

count    7.834323e+07
mean     7.775974e-01
std      2.550437e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.640000e+02
Name: tips, dtype: float64

In [41]:
%%time

ddf.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 407 ms, sys: 43.2 ms, total: 450 ms
Wall time: 49.7 s


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


In [49]:
m = ddf2.memory_usage_per_partition(deep=True).compute()
m

0       972629
1       973136
2       972947
3       972674
4       972962
         ...  
9956    945177
9957    945126
9958    945009
9959    944949
9960    945048
Length: 9961, dtype: int64

In [45]:
m.apply(dask.utils.format_bytes)

0       2.10 MiB
1       2.11 MiB
2       2.10 MiB
3       2.10 MiB
4       2.10 MiB
          ...   
9956    2.05 MiB
9957    2.05 MiB
9958    2.05 MiB
9959    2.04 MiB
9960    2.04 MiB
Length: 9961, dtype: object

In [50]:
m.apply(dask.utils.format_bytes)

0       0.93 MiB
1       0.93 MiB
2       0.93 MiB
3       0.93 MiB
4       0.93 MiB
          ...   
9956    0.90 MiB
9957    0.90 MiB
9958    0.90 MiB
9959    0.90 MiB
9960    0.90 MiB
Length: 9961, dtype: object

In [48]:
%%time
# w/ pyarrow strings
ddf2.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 197 ms, sys: 33.1 ms, total: 230 ms
Wall time: 36.7 s


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


In [32]:
ddf2 = ddf.repartition("10 MB")
ddf2 = ddf2.persist()

In [33]:
%%time
# w/ pyarrow strings + good partitions
ddf2.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 47.1 ms, sys: 5.42 ms, total: 52.6 ms
Wall time: 1.36 s


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


In [34]:
ddf3 = ddf.repartition("100 MB")
ddf3 = ddf3.persist()

In [35]:
%%time
# w/ pyarrow strings + good partitions
ddf3.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 32.5 ms, sys: 3.59 ms, total: 36 ms
Wall time: 449 ms


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


In [21]:
x = ddf.memory_usage_per_partition().compute()
x

0       972129
1       972636
2       972447
3       972174
4       972462
         ...  
9956    944677
9957    944626
9958    944509
9959    944449
9960    944548
Length: 9961, dtype: int64

In [22]:
import dask
x.apply(dask.utils.format_bytes)

0       0.93 MiB
1       0.93 MiB
2       0.93 MiB
3       0.93 MiB
4       0.93 MiB
          ...   
9956    0.90 MiB
9957    0.90 MiB
9958    0.90 MiB
9959    0.90 MiB
9960    0.90 MiB
Length: 9961, dtype: object

In [19]:
%%time

ddf = ddf.persist()

CPU times: user 97.5 ms, sys: 6.8 ms, total: 104 ms
Wall time: 106 ms


In [16]:
client.restart()

<Client: 'tls://10.0.41.117:8786' processes=0 threads=0, memory=0 B>

# File format

In [47]:
%%time

import dask.dataframe as dd

ddf2 = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")

CPU times: user 1.41 s, sys: 49.5 ms, total: 1.46 s
Wall time: 3.25 s


In [ ]:
%%time

# # Convert to datetime
# ddf["on_scene_datetime"] = dd.to_datetime(ddf["on_scene_datetime"], format="mixed")
# ddf["pickup_datetime"] = dd.to_datetime(ddf["pickup_datetime"], format="mixed")

# Unpack columns
ddf = ddf.assign(
    accessible_vehicle=ddf.on_scene_datetime.isnull(),
    pickup_month=ddf.pickup_datetime.dt.month,
    pickup_dow=ddf.pickup_datetime.dt.dayofweek,
    pickup_hour=ddf.pickup_datetime.dt.hour,
)
ddf = ddf.drop(columns=["on_scene_datetime", "pickup_datetime"])

# Format airport_fee
ddf["airport_fee"] = ddf["airport_fee"].replace("None", 0).astype(float).fillna(0)

# Remove outliers
lower_bound = 0
Q3 = ddf["trip_time"].quantile(0.75)
upper_bound = Q3 + (1.5 * (Q3 - lower_bound))
ddf = ddf.loc[(ddf["trip_time"] >= lower_bound) & (ddf["trip_time"] <= upper_bound)]

# # Categorize *_flag columns
# ddf = ddf.categorize(columns=["shared_request_flag", "shared_match_flag", "access_a_ride_flag", "wav_request_flag", "wav_match_flag"])

# ddf.to_parquet(...)

# Partition size

# Data types

Look at a dashboard plot (maybe the GIL contention plot?) that demostrates we could benefit from PyArrow strings

In [46]:
dask.config.set({"dataframe.convert-string": True})

In [ ]:
%%time

# # Convert to datetime
# ddf["on_scene_datetime"] = dd.to_datetime(ddf["on_scene_datetime"], format="mixed")
# ddf["pickup_datetime"] = dd.to_datetime(ddf["pickup_datetime"], format="mixed")

# Unpack columns
ddf = ddf.assign(
    accessible_vehicle=ddf.on_scene_datetime.isnull(),
    pickup_month=ddf.pickup_datetime.dt.month,
    pickup_dow=ddf.pickup_datetime.dt.dayofweek,
    pickup_hour=ddf.pickup_datetime.dt.hour,
)
ddf = ddf.drop(columns=["on_scene_datetime", "pickup_datetime"])

# Format airport_fee
ddf["airport_fee"] = ddf["airport_fee"].replace("None", 0).astype(float).fillna(0)

# Remove outliers
lower_bound = 0
Q3 = ddf["trip_time"].quantile(0.75)
upper_bound = Q3 + (1.5 * (Q3 - lower_bound))
ddf = ddf.loc[(ddf["trip_time"] >= lower_bound) & (ddf["trip_time"] <= upper_bound)]

# # Categorize *_flag columns
# ddf = ddf.categorize(columns=["shared_request_flag", "shared_match_flag", "access_a_ride_flag", "wav_request_flag", "wav_match_flag"])

# ddf.to_parquet(...)

# Conclusions

- Explored Dask dashboard
    - Saw plots that are available
    - Learned to interpret them
- Best practices
    - File format, partition size, data types, etc.

In [10]:
import dask
(ddf.memory_usage_per_partition()).mean().compute()

type: _sum() got an unexpected keyword argument 'skipna'

In [12]:
x = ddf.head()
x

,Unnamed: 0,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,...,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,accessible_vehicle,pickup_month,pickup_dow,pickup_hour
0,542399,HV0005,B02510,NaN,2019-02-01 18:47:50,2019-02-01 19:04:05,37,36,1.21,599,...,9.07,N,Y,N,N,NaN,True,2,4,18
1,597094,HV0005,B02510,NaN,2019-02-01 19:32:45,2019-02-01 19:54:11,134,82,6.20,1079,...,16.11,N,Y,N,N,NaN,True,2,4,19
2,642693,HV0005,B02510,NaN,2019-02-01 19:46:18,2019-02-01 20:16:30,37,145,5.00,1559,...,16.38,N,Y,N,N,NaN,True,2,4,19
3,741280,HV0003,B02872,B02872,2019-02-01 20:57:31,2019-02-01 21:06:41,179,179,0.89,383,...,5.76,N,N,N,N,NaN,False,2,4,21
4,823961,HV0003,B02887,B02887,2019-02-01 23:08:35,2019-02-01 23:34:58,234,265,4.99,1428,...,40.75,N,N,N,N,NaN,False,2,4,23


In [13]:
x.dtypes

Unnamed: 0                 int64
hvfhs_license_num         object
dispatching_base_num      object
originating_base_num      object
request_datetime          object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
trip_miles               float64
trip_time                  int64
base_passenger_fare      float64
tolls                    float64
bcf                      float64
sales_tax                float64
congestion_surcharge     float64
airport_fee              float64
tips                     float64
driver_pay               float64
shared_request_flag     category
shared_match_flag       category
access_a_ride_flag      category
wav_request_flag        category
wav_match_flag          category
accessible_vehicle          bool
pickup_month               int32
pickup_dow                 int32
pickup_hour                int32
dtype: object

In [ ]:
dask.config.set({"dataframe.convert-string": True})

## read csv data

In [ ]:
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
                     dtype = {
                     "shared_request_flag": "category",
                     "shared_match_flag": "category",
                     "access_a_ride_flag": "category",
                     "wav_request_flag": "category",
                     "wav_match_flag": "category",    
                     }, 
                     parse_dates=["request_datetime", "on_scene_datetime", # this is not working well see error below
                                  "pickup_datetime", "dropoff_datetime"]
                    )

In [ ]:
#notice the ints and floats are infered as 64
df_csv.dtypes

In [ ]:
#this is not working
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)

```python
ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

The following columns failed to properly parse as dates:

- on_scene_datetime

This is usually due to an invalid value in that column. To
diagnose and fix it's recommended to drop these columns from the
`parse_dates` keyword, and manually convert them to dates later
using `dd.to_datetime`.


```

## If we just do 
```python
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*",)
```

It will be infering date times and categories as strings, and the last category `wav_match_flag` because it has NaN is converting it to floats but the values are ["N", "Y"]

and this fails because of the dtype of `wav_match_flag`
```python
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)
```

## read parquet


In [46]:
%%time

df_pq = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")

CPU times: user 1.49 s, sys: 69.6 ms, total: 1.56 s
Wall time: 3.55 s


In [47]:
#beauty of parquet meta - get dtypes right
df_pq.dtypes

hvfhs_license_num       string[python]
dispatching_base_num    string[python]
originating_base_num    string[python]
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int32
DOLocationID                     int32
trip_miles                     float32
trip_time                        int32
base_passenger_fare            float32
tolls                          float32
bcf                            float32
sales_tax                      float32
congestion_surcharge           float32
airport_fee                    float32
tips                           float32
driver_pay                     float32
shared_request_flag           category
shared_match_flag             category
access_a_ride_flag            category
wav_request_flag              category
wav_match_flag                category
dtype: object

In [ ]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_pq.tips.sum().compute() / 1e6

In [ ]:
%%time
df_pq.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

### Try 100MB partitions

In [ ]:
df_pq = df_pq.repartition(partition_size="100MB").persist()

In [ ]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

## Went from ~ 5.9 s to 137ms (~43X faster)